# Estrategia del Momento Cuantitativo.

"Momento de Inversion" Se Refiere a la inversion in el stock que tiene el mejor incremento del precio.

Para este proyecto, iremos creando una estrategia de inversion que seleccione los stocks con  50 momentos de Precios Elevados. De alli, tendremos que calcular el trade recomendado por un Equal-Weight S&P 500 del portafolio de esos 50dias

# Importar Librerias

In [ ]:
import numpy as np 
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter


#  Importando Nuestro Stock

In [ ]:
stocks = pd.read_csv('SP500.csv')

from secrets import IEX_CLOUD_API_TOKEN

# Haciedo nuestra primera API

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json
data

# Parsing our API call

In [ ]:
data['yearChangePercent']

# Using Bach API Calls to improveperformance

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups(i)))
 #  print(symbol_strings(i))
#final_dataframe = pd.DataFrame(columns = my_colums)

#for symbol_string in symbol_strings:
#    print(symbol_string)
    
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [ ]:
final_dataframe = pd.DataFrame(columns = my_colums)
#final_dataframe

or symbol_strings in symbol_strings:
        batch_api_call_url = f'https://sandbox_iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json
        for symbol_strings in symbol_string.split(','):
            final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                data[symbol]['stats'][yearChangePercent],
                'N/A'
                
                
                
            ],
            index = my_columns),
                ignore_index = True
            )
final_dataframe


# Removiendo los stock de momento bajo

In [ ]:
final_dataframe.sort_value['One-Year Price Return', ascending = False, inplace = True]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

# Calcular El Number of Share to Buy

In [ ]:
def portfolio_input():
    global portfolio_size 
    portfolio_size = input('Enter the value of your portafolio')

    try:
        float(portfolio_size)
    except ValueError:
        print("That's not a number! \nPlease try again: ")
        portfolio_size = input ('Enter the value of your portfolio: ')
        
portfolio_input()
print(portfolio_size)
        

In [ ]:
#portafolio_size
position_size = portfolio_size/len(final_dataframe.index)
#print(position_size)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor[position_size/final_dataframe.loc[i, 'Price']]
number_of_apple_shares = position_size / 500
print(number_of_apple_shares)

# Crear un Modelo Estrategico de Momento Mas Real  

In [ ]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    
]

hqm_dataframe = pd.DataFrame(columns = hqm_colums)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox_iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats'][year1ChangePercent],
                'N/A',
                 data[symbol]['stats'][month6ChangePercent],
                'N/A',
                data[symbol]['stats'][month3ChangePercent],
                'N/A',
                data[symbol]['stats'][month1ChangePercent],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True                          
        )
hqm_dataframe

# Calculando Momentum Percentiles

In [ ]:
time_periods= [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
               ]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period}  Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe

# Calcular el HQM Score

In [ ]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

#  Seleccionar los 50 Momentum Stocks

In [ ]:
hqm_dataframe.sort_value['HQM Score', ascending = False, inplace = True]
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

# Calcular el Number of Share to Buy

In [ ]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor[position_size/hqm_dataframe.loc[i, 'Price']]
    
hqm_dataframe

# Formating our Excel Output

In [ ]:
writer = pd.ExcelWriter('momentums_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, 'Momentums Strategy', index = False)

# Crear los formatos que nesecitaremos para nuestra xlsxwriter

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border': 1
    }

)

dollar_template = writer.book.add_format(
    {
        'num_format' : '$0,00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border': 1
    }

)

integer_template = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border': 1
    }

)

percent_template = writer.book.add_format(
    {
        'num_format' : '0.0%',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border': 1
    }

)

In [ ]:
column_format = {
    
    'A' : ['Ticker', string_template],
    'B' : ['Price', dollar_template],
    'C' : ['Number of Shares to Buy', integer_template],
    'D' : ['One-Year Price Return', percent_template],
    'E' : ['One-Year Return Percentile', percent_template],
    'F' : ['Six-Month Price Return', percent_template],
    'G' : ['Six-Month Return Percentile', percent_template],
    'H' : ['Three-Month Price Return', percent_template],
    'I' : ['Three-Month Return Percentile', percent_template],
    'J' : ['One-Month Price Return', percent_template],
    'K' : ['One-Month Return Percentile', percent_template], 
    'L' : ['HQM Score', percent_template]
    
}


for column in column_format.keys():
    writer.sheet['Momentums Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheet['Momentums Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [ ]:
 writer.save()